In [ ]:
import jdata as jd
import numpy as np
import scipy.io
import pyvista as pv
import numpy as np
import vtk
import meshio
from fenics import *
from multiphenics import *
#filename = "/home/marius/Downloads/BrainWeb_Subject54.jmsh"
#outfile ="../meshes/BrainWeb_Subject54"
filename = "../meshes/MMC_Collins_Atlas_Mesh_Version_2L.mat"
outfile = "../meshes/MMC_Collins"
scaling = 1e-3 # mm to m

infile_dict_mat = {"CSF":2, "WM":3, "GM":4, "skull":1, "scalp":0, "rest":10}  
infile_dict_jmsh = {"CSF":3, "WM":4, "GM":5, "skull":2, "scalp":1, "rest":0}


fluid_id = 2
porous_id = 1

# boundary ids
interface_id = 1
rigid_skull_id = 2
spinal_outlet_id = 3
fix_stem_id = 4


def jmesh_to_vtk(infile, outfile, reduce=False, scaling=1.0):
    if infile.endswith(".jmsh"):
        jmesh = jd.load(infile)
        meshElem = jmesh["MeshElem"]
        points = jmesh["MeshVertex3"]*scaling
        infile_dict = infile_dict_jmsh
    elif infile.endswith(".mat"):
        mat_contents = scipy.io.loadmat(infile)
        points = mat_contents["node"]*scaling
        meshElem = mat_contents["elem"]
        infile_dict = infile_dict_mat
 
    subdomains = meshElem[:,4]
    
    if reduce:
        subdomains[subdomains==infile_dict["rest"]] = 10
        subdomains[subdomains==infile_dict["scalp"]] = 10
        subdomains[subdomains==infile_dict["skull"]] = 10
        subdomains[subdomains==infile_dict["CSF"]] = fluid_id
        subdomains[subdomains==infile_dict["WM"]] = porous_id
        subdomains[subdomains==infile_dict["GM"]] = porous_id
        
    n = meshElem.shape[0]
    p = 4 # number of points per cell for TETRA

    c = np.insert(meshElem[:,:4] - 1, 0, p, axis=1)
    cell_type = np.repeat(vtk.VTK_TETRA, n)
    offset = np.arange(start=0, stop=n*(p+1), step=p+1)
    grid = pv.UnstructuredGrid(offset, c, cell_type, points)
    grid.cell_arrays["subdomains"] = subdomains.flatten()
    if reduce:
        grid = grid.threshold(5, scalars="subdomains", invert=True)
    grid.save(outfile)
    

In [ ]:
jmesh_to_vtk(filename, outfile + ".vtk", reduce=False)
mesh = meshio.read(outfile + ".vtk")
meshio.write(outfile + ".xdmf", mesh)

jmesh_to_vtk(filename, outfile + "_reduced.vtk", reduce=True, scaling=scaling)
mesh = meshio.read(outfile + "_reduced.vtk")
meshio.write(outfile + "_reduced.xdmf", mesh)

In [ ]:
file = XDMFFile(outfile + "_reduced.xdmf")
mesh = Mesh()
file.read(mesh)

mf = MeshFunction("size_t", mesh, 3, 0)
file.read(mf, "subdomains")
boundaries = MeshFunction("size_t", mesh, 2, 0)
z_bottom = mesh.coordinates()[:,2].min()

In [ ]:
tdim = mesh.topology().dim()
mesh.init(tdim-1, tdim)
# set rigid skull boundary
rigid_skull = CompiledSubDomain("on_boundary")
rigid_skull.mark(boundaries, rigid_skull_id)

ztol = 5*1e-2
x_mid = 0.088
y_mid = 0.065
r = 0.01

def spinal_outlet(point):
    x,y,z = point[:]
    crit = ((x - x_mid)**2 + (y - y_mid)**2 < r**2) and z < 0.05
    return crit


# set internal interface
for f in facets(mesh):
    domains = []
    for c in cells(f):
        domains.append(mf[c])

    domains = list(set(domains))
    if len(domains) > 1:
        boundaries[f] = interface_id
    elif f.exterior():
        if domains[0] == fluid_id and spinal_outlet(f.midpoint().array()):
            boundaries[f] = spinal_outlet_id
        elif (domains[0] == porous_id and np.isclose(f.midpoint().array()[2], z_bottom, atol=ztol)):
            boundaries[f] = fix_stem_id

        
boundaries_outfile = XDMFFile(outfile + "_boundaries.xdmf")

boundaries_outfile.write(boundaries)
boundaries_outfile.close()

In [ ]:
def generate_subdomain_restriction(mesh, subdomains, subdomain_id):
    D = mesh.topology().dim()
    # Initialize empty restriction
    restriction = MeshRestriction(mesh, None)
    for d in range(D + 1):
        mesh_function_d = MeshFunction("bool", mesh, d)
        mesh_function_d.set_all(False)
        restriction.append(mesh_function_d)
    # Mark restriction mesh functions based on subdomain id
    for c in cells(mesh):
        if subdomains[c] == subdomain_id:
            restriction[D][c] = True
            for d in range(D):
                for e in entities(c, d):
                    restriction[d][e] = True
    # Return
    return restriction

fluid_restr = generate_subdomain_restriction(mesh, mf, fluid_id)
porous_restr = generate_subdomain_restriction(mesh, mf, porous_id)
fluid_restr._write(outfile + "_fluid.rtc.xdmf")
porous_restr._write(outfile + "_porous.rtc.xdmf")


In [ ]:
# evaluate volume shares:

dx = Measure("dx", domain=mesh, subdomain_data=mf)
csf_vol = assemble(1.0*dx(fluid_id))
parenchyma_vol = assemble(1.0*dx(porous_id))
print(f"CSF volume: {csf_vol*1e3} l")
print(f"Parenchyma volume: {parenchyma_vol*1e3} l")

In [ ]:
f.midpoint().array()